In [1]:
import os

In [2]:
%pwd

'/home/gaurav/ds study/Study/Text-Summerization/notebooks'

In [3]:

os.chdir('../')
%pwd

'/home/gaurav/ds study/Study/Text-Summerization'

In [4]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories


In [7]:
# configurationManager
class configurationManager:
    def __init__(self, config_file_path= CONFIG_FILE_PATH, param_file_path= PARAMS_FILE_PATH):
        
        self.config= read_yaml(config_file_path)
        self.param= read_yaml(param_file_path)

        if not os.path.exists(self.config.artifacts_root):
            create_directories(self.config.artifacts_root)

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config= self.config.data_transformation

        if not os.path.exists(self.config.root_dir):
            create_directories(self.config.root_dir)

        data_tranformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name
        )

        return data_tranformation_config


In [10]:
import os
from  textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


In [17]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config= config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,'samsum_dataset'))
        

In [18]:
try:
    config= configurationManager()
    data_tranformation_config = config.get_data_transformation_config()
    data_tranformation = DataTransformation(config=data_tranformation_config)
    data_tranformation.convert()
except Exception as e:
    raise e

[2024-01-16 20:58:48,104: INFO: common: ymal file:config/config.yaml load successfully]
[2024-01-16 20:58:48,107: INFO: common: ymal file:params.yaml load successfully]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/gaurav/study/anaconda3/envs/texts/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 81516.33 examples/s]
